In [30]:
!pip install numpy

We will take a neural network that takes two inputs, has one hidden layer with two nodes, and an output layer with one node.
<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width="600px">


We have 2 inputs, 6 weights, and 3 biases. Let's start by randomly initializing the weights and the biases in the netowrk.

In [31]:
import numpy as np

weights = np.around(np.random.uniform(size=6), decimals=2)
biases = np.around(np.random.uniform(size=3), decimals=2)

In [32]:
print(weights)
print(biases)

[0.47 0.04 0.08 0.73 0.64 0.03]
[0.3  0.22 0.06]


Now that we have the weights and biases for the network, let's compute the output for a given input x1 and x2.

In [33]:
x1 = 0.5
x2 = 0.85

Let's compute z11 and z12, the weighted sum of the inputs.

In [34]:
z11 = x1 * weights[0] + x2 * weights[1] + biases[0]
z12 = x1 * weights[2] + x2 * weights[3] + biases[1]
print("z11 =",z11,"and z12 =",z12)

z11 = 0.569 and z12 = 0.8805


Next, let's assume we use the sigmoid function as the activation function and compute the activation of the nodes a11 and a12, from the hidden layer.

In [35]:
a11 = 1.0 / (1.0 + np.exp(-z11))
a12 = 1.0 / (1.0 + np.exp(-z12))

print("The activation function of the first node in the hidden layer is",np.around(a11, decimals=4),"and the second is",np.around(a12, decimals=4))

The activation function of the first node in the hidden layer is 0.6385 and the second is 0.7069


These activations will server as inputs to the output layer. Let's compute the weighted sum of these inputes to the node in the output layer, z2.

In [36]:
z2 = a11 * weights[4] + a12 * weights[5] + biases[2]
print("The weighted sum of the inputs at the node in the output layer is",np.around(z2, decimals=4))

The weighted sum of the inputs at the node in the output layer is 0.4899


Finally, let's compute the output of the network as the activation of the node in the output layer, a2.

In [37]:
a2 = 1.0 / (1.0 + np.exp(-z2))

print("The output of the network is",a2)

The output of the network is 0.6200754560836037


Obviously, neural networks for real problems are composed of many hidden layers and many more nodes in each layer. So, we can't continue making predictions using this very inefficient approach of computing the weighted sum at each node and the activation of each node manually.

In order to code an automatic way of making predictions, let's generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, but we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width="600px">


# Initialize a Network
Let's start by formally defining the structure of the network.

In [38]:
n = 2 # number of inputes
num_hidden_layer = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

Now that we defined the structure of the network, let's go ahead and inititailize the weights and the biases in the network to random numbers. In order to be able to initialize the weights and the biases to random numbers.


In [39]:
import numpy as np

num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network as an empty dictionary

for layer in range(num_hidden_layer + 1):

  #determine the name of the layer
  if layer == num_hidden_layer:
    layer_name = 'output'
    num_nodes = num_nodes_output
  else:
    layer_name = 'layer_'+str(layer+1)
    num_nodes = m[layer]

  # initialize weights and biases associated with each node in the current layer
  network[layer_name] = {}
  for node in range(num_nodes):
    node_name = 'node_1'+str(node+1)
    network[layer_name][node_name] = {
        'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
        'bias': np.around(np.random.uniform(size=1), decimals=2),
    }

  num_nodes_previous = num_nodes

print(network)

{'layer_1': {'node_11': {'weights': array([0.52, 0.42]), 'bias': array([0.05])}, 'node_12': {'weights': array([0.57, 0.8 ]), 'bias': array([0.11])}}, 'layer_2': {'node_11': {'weights': array([0.28, 0.64]), 'bias': array([0.49])}, 'node_12': {'weights': array([0.51, 0.46]), 'bias': array([0.89])}}, 'output': {'node_11': {'weights': array([0.61, 0.6 ]), 'bias': array([0.44])}}}


Now let's put the above code in a function so that we are able to repetitively execute all this code whenever we want to construct a neural network.

In [40]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):

    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}

    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):

        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer]

        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }

        num_nodes_previous = num_nodes

    return network # return the network

Let's create a network that takes 5 inputes, has three hidden layers, had 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer and has 1 node in the output layer.

In [41]:
small_network = initialize_network(5, 3, [3, 2, 3], 1)

## Compute Weighted Sum at Each Node
The weighted sum at each node is computed as the dot product of the inputs and the weights plus the bias. So let's create a function called *compute_weighted_sum* that does just that.



In [42]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

Let's generate 5 inputs that we can feed to the **small_network**

In [43]:
from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


Compute the weighted sum at the first node in the first hidden layer.

In [44]:
weighted_sum = compute_weighted_sum(inputs, small_network['layer_1']['node_1']['weights'], small_network['layer_1']['node_1']['bias'])

## Compute Node Activation
Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.


In [45]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

#### Use the *node_activation* function to compute the output of the first node in the first hidden layer.


In [46]:
node_output = node_activation(weighted_sum)
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))


The output of the first node in the hidden layer is 0.8681


## Forward Propagation
The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the *compute_weighted_sum* and *node_activation* functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

The way we are going to accomplish this is through the following procedure:

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
5. Repeat steps 2 - 4 until we compute the output of the output layer.


In [47]:
def forward_propagate(network, inputs):

    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer

    for layer in network:

        layer_data = network[layer]

        layer_outputs = []
        for layer_node in layer_data:

            node_data = layer_data[layer_node]

            # compute the weighted sum and the output of each node at the same time
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))

        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))

        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

In [48]:
pred = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(pred[0], decimals=4)))


The outputs of the nodes in hidden layer number 1 is [0.8681, 0.8295, 0.803]
The outputs of the nodes in hidden layer number 2 is [0.9425, 0.9039]
The outputs of the nodes in hidden layer number 3 is [0.7267, 0.8477, 0.7605]
The predicted value by the network for the given input is 0.7961


So now we are done, we built the code to define a neural network. We can specify the number of inputs that a neural network can take, the number of hidden layers as well as the number of nodes in each hidden layer, and the number of nodes in the output layer.

We first use the *initialize_network* to create our neural network and define its weights and biases.


In [49]:
my_network = initialize_network(5, 3, [2, 3, 2], 3)

Then, for a given input,


In [50]:
inputs = np.around(np.random.uniform(size=5), decimals=2)

we compute the network predictions.


In [51]:
predictions = forward_propagate(my_network, inputs)
print('The predicted values by the network for the given input are {}'.format(predictions))

The outputs of the nodes in hidden layer number 1 is [0.8857, 0.8889]
The outputs of the nodes in hidden layer number 2 is [0.7822, 0.6965, 0.7411]
The outputs of the nodes in hidden layer number 3 is [0.868, 0.881]
The predicted values by the network for the given input are [0.8952, 0.8222, 0.8035]
